## Removing silent segments from recording

In [1]:
# imports
import IPython
from pydub import AudioSegment
import numpy as np
from scipy.io import wavfile
from plotly.offline import init_notebook_mode
import plotly.graph_objs as go
import plotly
import os

C:\Users\Lenovo\anaconda3\Lib\site-packages\pydub\utils.py:170: RuntimeWarning: Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work
  warn("Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work", RuntimeWarning)


In [3]:
# silence_removal.py
# ---------------------------------------------------------------
"""
Remove low‑energy (silent) 1‑second segments from every *.wav file
in `Initial/` and write the cleaned audio to `AfterSilent/`.
"""

import os
import glob
import numpy as np
from scipy.io import wavfile

# ─── paths ─────────────────────────────────────────────────────────
BASE_DIR = os.getcwd()                   # folder where this script lives
IN_DIR   = os.path.join(BASE_DIR, "Initial")
OUT_DIR  = os.path.join(BASE_DIR, "AfterSilent")
os.makedirs(OUT_DIR, exist_ok=True)      # create destination folder if missing

# ─── parameters you may tweak ─────────────────────────────────────
SEGMENT_SEC  = 1.0      # seconds per analysis window
ENERGY_RATIO = 0.5      # keep windows whose energy > 0.5 × median_energy

# ─── helper: process one wav file ─────────────────────────────────
def process_wav(in_path, out_path):
    fs, signal = wavfile.read(in_path)              # load
    signal = signal.astype(np.float32)
    if signal.dtype.kind == "i":                    # int16 ⇒ float‑1…1
        signal /= np.iinfo(signal.dtype).max

    # split into fixed‑length windows
    win_len = int(round(SEGMENT_SEC * fs))
    if win_len == 0:
        raise ValueError("SEGMENT_SEC too small for sampling rate!")
    segments = [signal[i:i+win_len] for i in range(0, len(signal), win_len)]
    energies = np.array([(seg**2).mean() for seg in segments])
    threshold = ENERGY_RATIO * np.median(energies)

    voiced = [seg for seg, e in zip(segments, energies) if e > threshold]
    if not voiced:
        print(f"!! {os.path.basename(in_path)} has no voiced segments—skipped")
        return

    cleaned = np.concatenate(voiced)
    wavfile.write(out_path, fs, cleaned.astype(np.float32))
    print(f"✓ {os.path.basename(in_path)} → {os.path.basename(out_path)}")

# ─── batch over every WAV in Initial/ ─────────────────────────────
wav_files = glob.glob(os.path.join(IN_DIR, "*.wav"))
if not wav_files:
    print("No .wav files found in 'Initial/'. Nothing to do.")
    exit()

for wav_path in sorted(wav_files):
    fname   = os.path.basename(wav_path)
    outpath = os.path.join(OUT_DIR, fname.replace("_AUDIO", "_silent"))
    process_wav(wav_path, outpath)

print(f"\nAll done! Cleaned files saved in '{OUT_DIR}'.")


C:\Users\Lenovo\AppData\Local\Temp\ipykernel_7860\1118296751.py:25: WavFileWarning:

Reached EOF prematurely; finished at 20752044 bytes, expected 166016044 bytes from header.

C:\Users\Lenovo\AppData\Local\Temp\ipykernel_7860\1118296751.py:25: WavFileWarning:

Reached EOF prematurely; finished at 26364844 bytes, expected 210918444 bytes from header.

C:\Users\Lenovo\AppData\Local\Temp\ipykernel_7860\1118296751.py:25: WavFileWarning:

Reached EOF prematurely; finished at 24281644 bytes, expected 194252844 bytes from header.



✓ 300_AUDIO.wav → 300_silent.wav
✓ 301_AUDIO.wav → 301_silent.wav
✓ 302_AUDIO.wav → 302_silent.wav


C:\Users\Lenovo\AppData\Local\Temp\ipykernel_7860\1118296751.py:25: WavFileWarning:

Reached EOF prematurely; finished at 31529644 bytes, expected 252236844 bytes from header.

C:\Users\Lenovo\AppData\Local\Temp\ipykernel_7860\1118296751.py:25: WavFileWarning:

Reached EOF prematurely; finished at 25363244 bytes, expected 202905644 bytes from header.

C:\Users\Lenovo\AppData\Local\Temp\ipykernel_7860\1118296751.py:25: WavFileWarning:

Chunk (non-data) not understood, skipping it.



✓ 303_AUDIO.wav → 303_silent.wav
✓ 304_AUDIO.wav → 304_silent.wav
✓ 319_AUDIO.wav → 319_silent.wav
✓ 320_AUDIO.wav → 320_silent.wav
✓ 321_AUDIO.wav → 321_silent.wav
✓ Noise.wav → Noise.wav

All done! Cleaned files saved in 'C:\Users\Lenovo\OneDrive\Desktop\6th Sem Project\Audio Model\AfterSilent'.


C:\Users\Lenovo\AppData\Local\Temp\ipykernel_7860\1118296751.py:25: WavFileWarning:

Reached EOF prematurely; finished at 26300844 bytes, expected 210406444 bytes from header.

